## Regresión Logística

La **regresión logística** es un modelo estadístico utilizado para predecir una variable dependiente binaria (es decir, con dos posibles valores: 0 o 1) a partir de un conjunto de variables independientes. A diferencia de la regresión lineal, que predice valores continuos, la regresión logística estima la **probabilidad** de que una observación pertenezca a una clase.

La función principal que utiliza es la **función sigmoide**:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

donde $ z = X \cdot \beta $, y $ \beta $ representa los pesos del modelo. La salida de la sigmoide es un valor entre 0 y 1, que se interpreta como una probabilidad.

El modelo se entrena minimizando la **función de pérdida logarítmica (log-loss)**, la cual penaliza las predicciones incorrectas más severamente cuanto más seguras son.

La función de pérdida logarítmica, utilizada para entrenar modelos de regresión logística, se define como:

$$
\mathcal{L}(y, \hat{y}) = -\frac{1}{m} \sum_{i=1}^{m} \left[ y^{(i)} \log(\hat{y}^{(i)}) + (1 - y^{(i)}) \log(1 - \hat{y}^{(i)}) \right]
$$

donde:

- $ m $ es el número total de ejemplos,
- $ y^{(i)} $ es la etiqueta verdadera (0 o 1),
- $ \hat{y}^{(i)} $ es la probabilidad predicha para la clase positiva,
- $ \log $ es el logaritmo natural.

Esta función mide qué tan bien se ajustan las probabilidades predichas a las etiquetas reales, penalizando con mayor severidad las predicciones incorrectas y seguras.

La regresión logística es ampliamente utilizada en clasificación binaria, como detección de fraudes, diagnóstico médico, y predicción de abandono de clientes.

## Proyecto: Predicción de Riesgo de Hospitalización

El **Hospital Regional "Salud Perfecta"** busca anticipar qué pacientes tienen un **alto riesgo de hospitalización** en los próximos 6 meses. Para ello, el equipo de ciencia de datos debe desarrollar un modelo predictivo usando **exclusivamente regresión logística** como algoritmo base.

### Objetivo
Construir un modelo de **clasificación binaria** que prediga la variable objetivo (`target`), la cual indica si un paciente será hospitalizado o no en el futuro cercano.

### Datos
El conjunto de datos incluye **15 variables predictoras** por paciente, que pueden abarcar factores clínicos, demográficos y hábitos de vida.

In [46]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [47]:
data = pd.read_csv("data/train_df.csv")


## Variables del Conjunto de Datos

El conjunto de datos contiene las siguientes **15 variables predictoras** más la variable objetivo (`target`). A continuación se listan junto con su tipo de dato:

| Variable                  | Tipo de dato |
|---------------------------|--------------|
| ratio_colesterol          | float64      |
| actividad_fisica          | float64      |
| presion_arterial          | float64      |
| nivel_glucosa             | float64      |
| indice_masa_corporal      | float64      |
| horas_sueno               | float64      |
| historial_diabetes        | int64        |
| frecuencia_cardiaca       | float64      |
| proteina_c_reactiva       | float64      |
| dias_ultima_consulta      | int64        |
| consumo_alcohol           | float64      |
| edad                      | int64        |
| nivel_estres              | float64      |
| genero_M                  | bool         |
| target                    | int64        |

- La variable `target` representa si un paciente fue o no hospitalizado en los próximos 6 meses.
- La variable `genero_M` es una codificación binaria del género (1 para masculino, 0 para femenino).

In [48]:
print(data["target"].value_counts(normalize=True))

target
0    0.504667
1    0.495333
Name: proportion, dtype: float64


Vemos que la data está muy bien balanceada. La proporción de hospitalizados y no hospitalizados es casi 50/50

In [49]:
print(data.corr(numeric_only=True)["target"].sort_values(ascending=False))

target                  1.000000
horas_sueno             0.019934
consumo_alcohol         0.018628
proteina_c_reactiva     0.009160
ratio_colesterol        0.005497
nivel_estres            0.001612
actividad_fisica        0.001469
frecuencia_cardiaca    -0.000867
dias_ultima_consulta   -0.002478
edad                   -0.004104
indice_masa_corporal   -0.016382
nivel_glucosa          -0.019268
historial_diabetes     -0.033649
presion_arterial       -0.038872
Name: target, dtype: float64


Tras calcular la correlación de Pearson entre las variables predictoras y la variable objetivo `target`, observamos que:

- Ninguna variable tiene una correlación fuerte (ni siquiera moderada) con `target`.
- Las correlaciones son muy cercanas a cero, tanto positivas como negativas.

#### ¿Qué significa esto?

- No existen relaciones lineales claras entre las variables predictoras y el riesgo de hospitalización.
- Sin embargo, **esto no implica que las variables no sean útiles**:
  - La **regresión logística no requiere correlaciones lineales fuertes** para funcionar bien.
  - Pueden existir relaciones **no lineales o combinaciones** de variables que sí tengan valor predictivo.

Por tanto, aún con baja correlación individual, el modelo puede encontrar patrones complejos para predecir adecuadamente la variable objetivo.

## Procesamiento de datos

In [50]:
# Eliminamos la columna 'paciente_id' del conjunto de datos, ya que no se utilizará en el modelo.
data = data.drop(columns=["paciente_id"])

# Convertimos las variables categóricas en variables binarias utilizando one-hot encoding.
# 'drop_first=True' asegura que eliminamos una de las categorías para evitar la multicolinealidad.
data = pd.get_dummies(data, drop_first=True)

# Separamos las características (X) de la variable objetivo (y).
# 'target' es la variable objetivo, que se elimina de X.
X = data.drop(columns=["target"]).values.astype(np.float64)

# Normalizamos las características para que todas tengan el mismo rango, usando el escalador.
X = scaler.fit_transform(X)

# Extraemos la variable objetivo (y) y la convertimos a un formato adecuado.
y = data["target"].values.reshape(-1, 1).astype(np.float64)

## Modelo logístico

In [51]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_loss(X, y, weights):
    m = X.shape[0]
    z = X @ weights
    h = sigmoid(z)
    epsilon = 1e-15
    loss = -(1/m) * np.sum(y * np.log(h + epsilon) + (1 - y) * np.log(1 - h + epsilon))
    return loss

def train(X, y, lr=0.01, epochs=1000):
    m, n = X.shape
    weights = np.zeros((n, 1))
    losses = []

    for epoch in range(epochs):
        z = X @ weights
        h = sigmoid(z)
        gradient = (1/m) * (X.T @ (h - y))
        weights -= lr * gradient

        if epoch % 100 == 0:
            loss = compute_loss(X, y, weights)
            losses.append(loss)
            print(f"Epoch {epoch}, Loss: {loss:.4f}")

    return weights

def predict(X, weights, threshold=0.5):
    probs = sigmoid(X @ weights)
    return (probs >= threshold).astype(int)

## Entrenamiento y Predicción

In [52]:
weights = train(X, y, lr=0.01, epochs=1000)
y_pred = predict(X, weights).flatten()

Epoch 0, Loss: 0.6931
Epoch 100, Loss: 0.6923
Epoch 200, Loss: 0.6918
Epoch 300, Loss: 0.6915
Epoch 400, Loss: 0.6912
Epoch 500, Loss: 0.6911
Epoch 600, Loss: 0.6910
Epoch 700, Loss: 0.6909
Epoch 800, Loss: 0.6909
Epoch 900, Loss: 0.6909


## Evaluación del Modelo

### Métrica de Evaluación: F1-Score

La **F1-Score** es una métrica de evaluación utilizada principalmente en problemas de clasificación binaria, especialmente cuando las clases están desbalanceadas o cuando tanto los **falsos positivos** como los **falsos negativos** son importantes.

#### ¿Qué mide?

El F1-Score es la **media armónica** entre la **precisión** (*precision*) y la **recuperación** (*recall*):

$$
\text{F1} = 2 \cdot \frac{\text{Precisión} \cdot \text{Recall}}{\text{Precisión} + \text{Recall}}
$$

- **Precisión (Precision)**: Qué proporción de las predicciones positivas fueron realmente positivas.  
  $$
  \text{Precisión} = \frac{TP}{TP + FP}
  $$

- **Recall (Sensibilidad)**: Qué proporción de los positivos reales fueron correctamente identificados.  
  $$
  \text{Recall} = \frac{TP}{TP + FN}
  $$

Donde:
- `TP`: Verdaderos positivos
- `FP`: Falsos positivos
- `FN`: Falsos negativos

#### ¿Por qué usar F1-Score?

- Es útil cuando hay un **desequilibrio entre clases** o cuando **los errores tienen diferentes costos**.
- Penaliza los modelos que tienen **alta precisión pero bajo recall** (o viceversa).
- Es más informativa que la simple precisión o la exactitud (accuracy) en muchos casos clínicos.

#### En este proyecto

Dado que queremos predecir **riesgo de hospitalización**, tanto **no detectar un paciente que sí será hospitalizado** como **alertar falsamente a un paciente sano** son errores críticos.  
El F1-Score proporciona un **equilibrio** adecuado entre estos dos tipos de errores y es una métrica adecuada para evaluar el modelo de regresión logística.

In [54]:
from sklearn.metrics import f1_score
print("F1-Score:", f1_score(y, y_pred))

F1-Score: 0.5314500110350916


## Conclusión

El modelo de regresión logística desarrollado para predecir el riesgo de hospitalización arrojó un **F1-Score de 0.5315**.

### Interpretación:

Este valor indica un **desempeño moderado** del modelo, con un balance razonable entre precisión y recall. Aunque el modelo logra identificar correctamente algunos pacientes en riesgo, **aún hay margen significativo de mejora**.

Dado que el F1-Score está solo ligeramente por encima de 0.5, esto sugiere que el modelo **no es mucho mejor que una predicción aleatoria**. Esto sugiere que el modelo puede mejorar. Para esto, se puede considerar:

- Explorar **transformaciones no lineales** de variables o interacciones entre ellas.
- Evaluar la selección de características y eliminar variables irrelevantes.
- Ajustar hiperparámetros como la tasa de aprendizaje o el número de iteraciones.
- Probar técnicas de **regularización** para mejorar la generalización.

Este resultado es un punto de partida útil, pero se requieren mejoras adicionales para que el modelo sea confiable en un entorno clínico real.